In [15]:
import scipy.io
import torch
import torch.nn as nn
import pandas as pd
import cv2
import numpy as np

import imutils
import dlib
from imutils import face_utils


In [12]:
d_path = "D:\\Dataset\\P01\\p01_loc1.mat"
predictor_path = "D:\Dataset\shape_predictor_68_face_landmarks (1).dat"
data = scipy.io.loadmat(d_path)


In [36]:
data['gaze_dirs'][0]


array([-0.12391766,  0.15204414])

In [39]:
dataset = []

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)


for img_ind in range(len(data['gaze_dirs'])):

	datapoint = ["full_face", "left_eye", "right_eye", "pitch_yaw_groundtruth", "pid" ]

	image = data['face_img'][img_ind]
	resized_face = cv2.resize(image, (120, 120), interpolation= cv2.INTER_LINEAR)

	datapoint[0] = resized_face
	datapoint[3] = data['gaze_dirs'][img_ind]

	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	# detect faces in the grayscale image
	rects = detector(gray, 1)

	for (rect_ind, rect) in enumerate(rects):
		# determine the facial landmarks for the face region, then
		# convert the landmark (x, y)-coordinates to a NumPy array
		shape = predictor(gray, rect)
		shape = face_utils.shape_to_np(shape)
		# loop over the face parts individually

		for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
			# clone the original image so we can draw on it, then
			# display the name of the face part on the image
			clone = image.copy()

			(x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
			roi = image[y:y + h, x:x + w]
			roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
			resized_part = cv2.resize(roi, (80, 120), interpolation= cv2.INTER_LINEAR)

			if name == "left_eye":
				datapoint[1] = resized_part
			elif name == "right_eye":
				datapoint[2] = resized_part

	dataset.append(np.array(datapoint, dtype=object))
	print("Done with : %d/%d" % (img_ind, len(data['gaze_dirs'])))
		



Done with : 0/2670
Done with : 1/2670
Done with : 2/2670
Done with : 3/2670
Done with : 4/2670
Done with : 5/2670
Done with : 6/2670
Done with : 7/2670
Done with : 8/2670
Done with : 9/2670
Done with : 10/2670
Done with : 11/2670
Done with : 12/2670
Done with : 13/2670
Done with : 14/2670
Done with : 15/2670
Done with : 16/2670
Done with : 17/2670
Done with : 18/2670
Done with : 19/2670
Done with : 20/2670
Done with : 21/2670
Done with : 22/2670
Done with : 23/2670
Done with : 24/2670
Done with : 25/2670
Done with : 26/2670
Done with : 27/2670
Done with : 28/2670
Done with : 29/2670
Done with : 30/2670
Done with : 31/2670
Done with : 32/2670
Done with : 33/2670
Done with : 34/2670
Done with : 35/2670
Done with : 36/2670
Done with : 37/2670
Done with : 38/2670
Done with : 39/2670
Done with : 40/2670
Done with : 41/2670
Done with : 42/2670
Done with : 43/2670
Done with : 44/2670
Done with : 45/2670
Done with : 46/2670
Done with : 47/2670
Done with : 48/2670
Done with : 49/2670
Done with 

In [40]:
np.save("dataset.npy", np.array(dataset), allow_pickle=True, fix_imports=True)